In [7]:
import numpy as np

In [8]:
!pip install torch

In [9]:
import pandas as pd

In [10]:
path = '/dgxa_home/se22uari173/train.csv'

In [11]:
df = pd.read_csv(path)

In [12]:
df

,id,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance
...,...,...,...,...
12455,train_12455,#Person1#: Excuse me. You are Mr. Green from M...,Tan Ling picks Mr. Green up who is easily reco...,pick up someone
12456,train_12456,#Person1#: Mister Ewing said we should show up...,#Person1# and #Person2# plan to take the under...,conference center
12457,train_12457,#Person1#: How can I help you today?\n#Person2...,#Person2# rents a small car for 5 days with th...,rent a car
12458,train_12458,#Person1#: You look a bit unhappy today. What'...,#Person2#'s mom lost her job. #Person2# hopes ...,job losing


In [13]:
!pip install spacy

In [14]:
import spacy

/dgxa_home/se22uari173/anaconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [15]:
nlp = spacy.load("en_core_web_sm")

/dgxa_home/se22uari173/anaconda3/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/dgxa_home/se22uari173/anaconda3/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [16]:
import torch

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
def preprocess_text(text):
    doc = nlp(text)
    cleaned_tokens = [
        token.text for token in doc if not token.is_punct and not token.is_space
    ]
    return " ".join(cleaned_tokens)

In [19]:
import numpy as np
percent = 97
print(np.percentile([len(x) for x in df['dialogue']],percent))

1584.0


In [20]:
print(np.percentile([len(x) for x in df['summary']],percent))

283.0


In [21]:
max_input_seq_length = 1584
max_output_seq_length = 290

In [22]:
dialogue_list = list(df['dialogue'])

In [23]:
summary_list = list(df['summary'])

In [24]:
for i in range(len(dialogue_list)):
    dialogue_list[i] = preprocess_text(dialogue_list[i])
for i in range(len(summary_list)):
    summary_list[i] = preprocess_text(summary_list[i])

In [25]:
maxi = 0
for i in dialogue_list:
    if(len(i)>maxi):
        maxi = len(i)

In [26]:
maxi

4877

In [27]:
def is_validlength(paragraph,max_seq_length):
    return(len(paragraph)<max_input_seq_length)
valid_textsummaries_idx = []
for i in range(len(df['summary'])):
    dialogue,summary = dialogue_list[i],summary_list[i]
    if(is_validlength(dialogue,max_input_seq_length) and is_validlength(summary,max_output_seq_length)):
        valid_textsummaries_idx.append(i)

In [28]:
len(valid_textsummaries_idx)

12180

In [29]:
dialogue_list_new = []
summary_list_new = []
for i in valid_textsummaries_idx:
    dialogue_list_new.append(dialogue_list[i])
    summary_list_new.append(summary_list[i])

In [30]:
from torch.utils.data import Dataset,DataLoader

In [31]:
class TextDataset(Dataset):
    def __init__(self,dialogues,summaries):
        self.dialogues = dialogues
        self.summaries = summaries
    def __len__(self):
        return len(self.summaries)
    def __getitem__(self,j):
        return self.dialogues[j],self.summaries[j]

In [32]:
data = TextDataset(dialogue_list_new,summary_list_new)

In [33]:
data = TextDataset(dialogue_list_new, summary_list_new)
batch_size = 1
train_dataset = DataLoader(data, batch_size=batch_size, shuffle=True)

In [34]:
c = 1
for batch in train_dataset:
    dialogues,summaries = batch
    print(summaries)
    if(c==1):
        break

('Adam Smith gives Lily Brown her umbrella They discover that they live on the same floor of the same building They introduce themselves',)


In [35]:
!pip install transformers

In [36]:
!pip install sentencepiece

In [38]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [39]:
len(train_dataset)

12180

In [40]:
max = 0
for i in dialogue_list_new:
    if(max<len(tokenizer(i)['input_ids'])):
        max = len(tokenizer(i)['input_ids'])
print(max)

433


In [41]:
max = 0
for i in summary_list_new:
    if(max<len(tokenizer(i)['input_ids'])):
        max = len(tokenizer(i)['input_ids'])
print(max)

96


In [42]:
len(batch)

2

In [43]:
input_ids = []
output_ids = []
max_input_seq_length = 450
max_output_seq_length = 450
for batch in train_dataset:
    if(len(batch)==2):
        dialogues, summaries = batch 
        tokenized_dialogues = tokenizer(list(dialogues),max_length=max_input_seq_length,padding='max_length',return_tensors='pt',add_special_tokens=False)
        tokenized_summaries = tokenizer(list(summaries),max_length=max_output_seq_length,padding='max_length',return_tensors='pt',add_special_tokens=True)
        input_ids.append(tokenized_dialogues['input_ids'])
        output_ids.append(tokenized_summaries['input_ids'])

In [44]:
tensor_tokenized_inputs = torch.cat(input_ids,dim=0)

In [45]:
tensor_tokenized_outputs = torch.cat(output_ids,dim=0)

In [46]:
import math
from torch import nn
import torch.nn.functional as func

In [47]:
class Multiheadattention(nn.Module):
    def __init__(self,weights_dim,n_heads):
        super().__init__()
        self.weights_dim = weights_dim
        self.n_heads = n_heads
        self.head_dimensions = weights_dim//n_heads
        self.qkv_matrices = nn.Linear(weights_dim,3*weights_dim)
        self.linear_layer = nn.Linear(weights_dim,weights_dim)
    def forward(self,x,mask):
        batch,maxseqlength,weight_dim = x.size()
        qkv = self.qkv_matrices(x)
        qkv = qkv.reshape(batch,maxseqlength,self.n_heads,3*self.head_dimensions)
        qkv = qkv.permute(0,2,1,3)
        q,k,v = qkv.chunk(3,dim=-1)
        k_transpose = k.transpose(-1,-2)
        product = torch.matmul(q,k_transpose)
        product = product/math.sqrt(q.size()[-1])
        if mask is not None:
            product = product+mask
        attention_scores = func.softmax(product,dim=-1)
        attentionintoV = torch.matmul(attention_scores,v)
        n = (self.n_heads*self.head_dimensions)
        output = self.linear_layer(attentionintoV.reshape(batch,maxseqlength,n))
        return output

In [48]:
class LayerNormalization(torch.nn.Module):
    def __init__(self,eps):
        super(LayerNormalization, self).__init__()
        self.eps = eps 
    def forward(self,inp):
        mean = inp.mean(dim=-1, keepdim=True) 
        var = ((inp - mean) ** 2).mean(dim=-1, keepdim=True)
        std = torch.sqrt(var + self.eps)
        y = (inp - mean) / std
        return y

In [49]:
class feedfowardlayer(nn.Module):
    def __init__(self,weights_dims,hidden_dim,drop_prob):
        super(feedfowardlayer, self).__init__()
        self.weights_dims = weights_dims
        self.hidden_dims = hidden_dim
        self.linearlayer1 = nn.Linear(self.weights_dims,self.hidden_dims)
        self.relu = nn.ReLU()
        self.linearlayer2 = nn.Linear(self.hidden_dims,self.weights_dims)
        self.drop_probability = drop_prob
        self.drop_neurons = nn.Dropout(p=self.drop_probability)
    def forward(self,x):
        out = self.linearlayer1(x)
        out = self.relu(out)
        out = self.drop_neurons(out)
        out = self.linearlayer2(out)
        return out

In [50]:
class Subencoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,drop_probability):
        super(Subencoder,self).__init__()
        self.attention = Multiheadattention(weight_dim,n_heads)
        self.norm = LayerNormalization(1e-6)
        self.dropout1 = nn.Dropout(p = drop_probability)
        self.ffd = feedfowardlayer(weight_dim,ff_hidden,drop_probability)
        self.norm2 = LayerNormalization(1e-6)
        self.dropout2 = nn.Dropout(p = drop_probability)
    def forward(self,x,mask):
        residual = x
        x = self.attention(x,mask)
        x = self.dropout1(x)
        x = self.norm(x+residual)
        residual = x
        x = self.ffd(x)
        x = self.dropout2(x)
        x = self.norm2(x+residual)
        return x

In [51]:
class Sequentialencoder(nn.Sequential):
    def foward(self,x,mask):
        self.x = x
        self.mask = mask
        for module in self._modules.values():
            x = module(self.x,self.mask)
        return y

In [52]:
class Encoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,layers,drop_probability):
        super(Encoder, self).__init__()
        encoded_layers = []
        for k in range(layers): 
            l = Subencoder(weight_dim, ff_hidden,n_heads,drop_probability)
            encoded_layers.append(l)
        self.layers = nn.ModuleList(encoded_layers)
    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return x

In [53]:
class Multiheadcrossattention(nn.Module):
    def __init__(self,weights_dim,n_heads):
        super().__init__()
        self.weights_dim = weights_dim
        self.n_heads = n_heads
        self.head_dimensions = weights_dim//n_heads
        self.kv_matrices = nn.Linear(weights_dim,2*weights_dim)
        self.q_matrix = nn.Linear(weights_dim,weights_dim)
        self.linear_layer = nn.Linear(weights_dim,weights_dim)
    def forward(self,y,x,mask):
        batches,maxseqlength,weight_dim = x.size()
        kv = self.kv_matrices(x)
        q = self.q_matrix(y)
        kv = kv.reshape(batches,maxseqlength,self.n_heads,2*self.head_dimensions)
        kv = kv.permute(0,2,1,3)
        k,v = kv.chunk(2,dim=-1)
        q = q.reshape(batches,maxseqlength,self.n_heads,self.head_dimensions)
        q = q.permute(0,2,1,3)
        k_transpose = k.transpose(-1,-2)
        product = torch.matmul(q,k_transpose)
        product = product/math.sqrt(q.size()[-1])
        if mask is not None:
            product = product+mask
        attention_scores = func.softmax(product,dim=-1)
        attentionintoV = torch.matmul(attention_scores,v)
        n = (self.n_heads*self.head_dimensions)
        output = self.linear_layer(attentionintoV.reshape(batches,maxseqlength,n))
        return output

In [54]:
class Maskedattention(nn.Module):
    def __init__(self,weights_dim,n_heads):
        super().__init__()
        self.weights_dim = weights_dim
        self.n_heads = n_heads
        self.head_dimensions = weights_dim//n_heads
        self.qkv_matrices = nn.Linear(weights_dim,3*weights_dim)
        self.linear_layer = nn.Linear(weights_dim,weights_dim)
    def forward(self,x,mask,padmask):
        batch,maxseqlength,weight_dim = x.size()
        qkv = self.qkv_matrices(x)
        qkv = qkv.reshape(batch,maxseqlength,self.n_heads,3*self.head_dimensions)
        qkv = qkv.permute(0,2,1,3)
        q,k,v = qkv.chunk(3,dim=-1)
        k_transpose = k.transpose(-1,-2)
        product = torch.matmul(q,k_transpose)
        product = product/math.sqrt(q.size()[-1])
        if mask is not None:
            product = product+mask+padmask
        attention_scores = func.softmax(product,dim=-1)
        attentionintoV = torch.matmul(attention_scores,v)
        n = (self.n_heads*self.head_dimensions)
        output = self.linear_layer(attentionintoV.reshape(batch,maxseqlength,n))
        return output

In [55]:
class Subdecoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,drop_probability):
        super(Subdecoder,self).__init__()
        self.maskedattention = Maskedattention(weight_dim,n_heads)
        self.norm = LayerNormalization(1e-6)
        self.dropout1 = nn.Dropout(p = drop_probability)
        self.Multicrossattention = Multiheadcrossattention(weight_dim,n_heads)
        self.norm2 = LayerNormalization(1e-6)
        self.dropout2 = nn.Dropout(p = drop_probability)
        self.ffd = feedfowardlayer(weight_dim,ff_hidden,drop_probability)
        self.norm3 = LayerNormalization(1e-6)
        self.dropout3 = nn.Dropout(p = drop_probability)
    def forward(self,x,y,mask,padmask,crossmask):
        residual = y
        y = self.maskedattention(y,mask,padmask)
        y = self.dropout1(y)
        y = self.norm(y+residual)
        residual = y
        y = self.Multicrossattention(y,x,crossmask)
        y = self.dropout2(y)
        y = self.norm2(y+residual)
        residual = y
        y = self.ffd(y)
        y = self.dropout3(y)
        y = self.norm3(y+residual)
        return y

In [56]:
class Sequentialdecoder(nn.Sequential):
    def forward(self,x,y,mask,padmask,crossattentpadmask):
        for module in self._modules.values():
            y = module(x,y,mask,padmask,crossattentpadmask)
        return y

In [57]:
class Decoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,layers,drop_probability):
        super().__init__()
        decoded_layers = []
        for k in range(layers):
            l = Subdecoder(weight_dim,ff_hidden,n_heads,drop_probability)
            decoded_layers.append(l)
        self.layers = Sequentialdecoder(*[Subdecoder(weight_dim,ff_hidden,n_heads,drop_probability)])
    def forward(self,x,y,mask,padmask,crossattentpadmask):
        resdecoder = self.layers(x,y,mask,padmask,crossattentpadmask)
        return resdecoder

In [58]:
vocab_size = tokenizer.vocab_size

In [59]:
vocab_size

32000

In [60]:
class PositionalEncodingWithFeatureTransformation(nn.Module):
    def __init__(self, input_dim, output_dim, max_sequence_length):
        super(PositionalEncodingWithFeatureTransformation, self).__init__()
        self.max_sequence_length = max_sequence_length
        self.output_dim = output_dim
        self.linear = nn.Linear(input_dim, output_dim)
        self.positional_embeddings = nn.Parameter(torch.zeros(max_sequence_length, output_dim), requires_grad=True)
        self.create_positional_encodings()
    def create_positional_encodings(self):
        even_i = torch.arange(0, self.output_dim, 2).float()
        denominator = torch.pow(10000, even_i / self.output_dim)
        position = torch.arange(self.max_sequence_length).reshape(self.max_sequence_length, 1)
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        self.positional_embeddings.data = torch.flatten(stacked, start_dim=1, end_dim=2)
    def forward(self, x):
        x = self.linear(x)  
        positional_encoded = self.positional_embeddings[:x.size(1), :].unsqueeze(0) 
        return x + positional_encoded

In [64]:
class Word2VecEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2VecEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
    def forward(self, token_ids):
        return self.embedding(token_ids)

In [65]:
def create_padding_mask(tensor_tokenized_inputs,batch_size,seq_len,n_heads):
    padding_mask = (tensor_tokenized_inputs != 0).float()
    padding_mask = padding_mask.unsqueeze(0)
    mask_2d = padding_mask.unsqueeze(1).expand(-1, n_heads, seq_len, seq_len)  
    padding_mask_expanded = (1.0 - mask_2d) * -3.4028e+38  
    padding_mask_expanded[mask_2d == 1.0] = 0  
    return padding_mask_expanded

In [66]:
def create_padding_mask_2(tensor_tokenized_outputs_, seq_len, n_heads):
    padding_mask = (tensor_tokenized_outputs_ == 0).float()
    repeated_mask_350 = padding_mask.repeat(seq_len, 1)
    repeated_mask_8 = repeated_mask_350.unsqueeze(0).unsqueeze(0).repeat(1, n_heads, 1, 1)  
    padding_mask_expanded = repeated_mask_8 * -3.4028e+38  
    padding_mask_expanded[padding_mask_expanded == 0] = 0  
    return padding_mask_expanded

In [67]:
def create_lookahead_mask(seq_len,n_heads):
    lookahead_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1)  
    lookahead_mask = lookahead_mask * -3.4028e+38  
    lookahead_mask = lookahead_mask + torch.tril(torch.zeros(seq_len, seq_len))
    lookahead_mask = lookahead_mask.unsqueeze(0).unsqueeze(0) 
    lookahead_mask_expanded = lookahead_mask.repeat(1, n_heads, 1, 1)  
    return lookahead_mask_expanded

In [68]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, weight_dim, ff_hidden_dim, n_heads, num_layers, max_input_seq_length,max_output_seq_length):
        super(Transformer, self).__init__()
        num_layers = int(num_layers) 
        self.embedding = nn.Embedding(vocab_size, weight_dim)
        self.positional_input_encoding = PositionalEncodingWithFeatureTransformation(32000,512,max_input_seq_length)
        self.positional_output_encoding = PositionalEncodingWithFeatureTransformation(32000,512,max_output_seq_length)
        self.encoder = Encoder(weight_dim, ff_hidden_dim, n_heads, num_layers, 0.1)
        self.decoder = Decoder(weight_dim, ff_hidden_dim, n_heads, num_layers, 0.1)
        self.fc_out = nn.Linear(weight_dim, vocab_size)
    def forward(self,src,tgt,src_mask,tgt_mask,lkahd_mask,crossattention_mask):
        enc_out = self.encoder(src,src_mask)
        dec_out = self.decoder(tgt,enc_out,lkahd_mask,tgt_mask,crossattention_mask)
        return self.fc_out(dec_out)

In [69]:
weight_dim = 512
ff_hidden_dim = 2048
n_heads = 8
num_layers = 6
vocab_size = tokenizer.vocab_size
max_seq_input_length = 450
max_seq_output_length = 450
transformer = Transformer(vocab_size,weight_dim, ff_hidden_dim, n_heads, num_layers,max_seq_input_length,max_seq_output_length)

In [70]:
transformer.to(device)

Transformer(
  (embedding): Embedding(32000, 512)
  (positional_input_encoding): PositionalEncodingWithFeatureTransformation(
    (linear): Linear(in_features=32000, out_features=512, bias=True)
  )
  (positional_output_encoding): PositionalEncodingWithFeatureTransformation(
    (linear): Linear(in_features=32000, out_features=512, bias=True)
  )
  (encoder): Encoder(
    (layers): ModuleList(
      (0-5): 6 x Subencoder(
        (attention): Multiheadattention(
          (qkv_matrices): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffd): feedfowardlayer(
          (linearlayer1): Linear(in_features=512, out_features=2048, bias=True)
          (relu): ReLU()
          (linearlayer2): Linear(in_features=2048, out_features=512, bias=True)
          (drop_neurons): Dropout(p=0.1, inplace=False)
    

In [71]:
embedding_matrix_inputs = nn.Linear(512,512)
embedding_matrix_outputs = nn.Linear(512,512)
positional_embeddings = PositionalEncodingWithFeatureTransformation(input_dim=512,output_dim=512,max_sequence_length=450)
optimizer = torch.optim.Adam(list(transformer.parameters()) + list(embedding_matrix_inputs.parameters()) + list(embedding_matrix_outputs.parameters()) + list(positional_embeddings.parameters()),lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
word2vec = Word2VecEmbedding(32000,512)
for i in range(30):
    weights_dim = 512
    ff_hidden = 2048
    n_heads = 8
    drop_probability = 0.15
    num_layers = 3
    batch_size = 1
    for i in range(len(train_dataset)):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        current_batch_inputs = tensor_tokenized_inputs[start_idx:end_idx]  
        current_batch_outputs = tensor_tokenized_outputs[start_idx:end_idx]
    with torch.no_grad(): 
        X = word2vec(current_batch_inputs) 
    with torch.no_grad():
        Y = word2vec(current_batch_outputs)
    embedded_X = embedding_matrix_inputs(X)
    embedded_Y = embedding_matrix_outputs(Y)
    positional_embeddingsx = positional_embeddings(X) 
    final_X = embedded_X + positional_embeddingsx
    positional_embeddingsy = positional_embeddings(Y) 
    final_Y = embedded_Y + positional_embeddingsy
    src_mask = create_padding_mask(current_batch_inputs,1,450,8)
    tgt_mask = create_padding_mask(current_batch_outputs,1,450,8)
    lkahd_mask = create_lookahead_mask(450,8)
    crossattention_mask = create_padding_mask_2(current_batch_outputs,450,8)
    logits = transformer(final_X,final_Y,src_mask,tgt_mask,lkahd_mask,crossattention_mask)
    probabilities = func.softmax(logits,dim=-1)
    predicted_tokens = torch.argmax(probabilities,dim=-1)
    predicted_text = tokenizer.decode(predicted_tokens[0].tolist())
    loss = criterion(logits.view(-1, logits.size(-1)), current_batch_outputs.view(-1))
    print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

tensor(10.4728, grad_fn=<NllLossBackward0>)
tensor(7.6795, grad_fn=<NllLossBackward0>)
tensor(5.9802, grad_fn=<NllLossBackward0>)
tensor(6.6960, grad_fn=<NllLossBackward0>)
tensor(4.5027, grad_fn=<NllLossBackward0>)
tensor(2.3640, grad_fn=<NllLossBackward0>)
tensor(1.2672, grad_fn=<NllLossBackward0>)
tensor(0.8248, grad_fn=<NllLossBackward0>)
tensor(0.4476, grad_fn=<NllLossBackward0>)
tensor(0.3167, grad_fn=<NllLossBackward0>)
tensor(0.2390, grad_fn=<NllLossBackward0>)
tensor(0.1874, grad_fn=<NllLossBackward0>)
tensor(0.1323, grad_fn=<NllLossBackward0>)
tensor(0.0837, grad_fn=<NllLossBackward0>)
tensor(0.0577, grad_fn=<NllLossBackward0>)
tensor(0.0337, grad_fn=<NllLossBackward0>)
tensor(0.0261, grad_fn=<NllLossBackward0>)
tensor(0.0194, grad_fn=<NllLossBackward0>)
tensor(0.0148, grad_fn=<NllLossBackward0>)
tensor(0.0121, grad_fn=<NllLossBackward0>)
tensor(0.0103, grad_fn=<NllLossBackward0>)
tensor(0.0085, grad_fn=<NllLossBackward0>)
tensor(0.0074, grad_fn=<NllLossBackward0>)
tensor(0.0

In [72]:
predicted_text

"Person1 invites Anna to come to a get together Anna agrees after she knows Jack won't come because they are on bad terms</s> together wo Anna knows terms come</s>after knows come  knows Annat Annat wo Anna on Anna Annas together  knows get1 get1 they they1 Jack to Person they to to to to to Person Person Person they Person get together get to to Person get to to on they  to get1 Person Person Person they Jack Person they Person Person together to to to Person Person to get they get aftert Jackt Jackt'tt Jack' knows Jackt Jack Jack Jackt Jack' Jackt'' they Jacktt to' knowstt knows' Jacktt't Jackt Jackt''ttt to Jack' Jack Jack'' to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to to come come come come knows come come  come come come come come come come come come come come comet come come come come come come come come come come come come come come come come come come come 

In [73]:
end_pos = predicted_text.find("</s>")
if end_pos != -1:
    truncated_text = predicted_text[:end_pos]
else:
    truncated_text = predicted_text 
cleaned_text = " ".join(truncated_text.split())
print(cleaned_text)

Person1 invites Anna to come to a get together Anna agrees after she knows Jack won't come because they are on bad terms


In [74]:
!pip install gtts

In [75]:
from gtts import gTTS

In [76]:
import os

In [77]:
!pip install pygame

In [78]:
!pip install pydub

In [ ]:
userinput_text = input("Enter paragraph: ")

In [ ]:
newtext = preprocess_text(userinput_text)

In [ ]:
newtext_tokens = tokenizer(newtext,max_length=max_input_seq_length,padding='max_length',return_tensors='pt',add_special_tokens=False)

In [ ]:
tensor_tokenized_newtext = torch.cat(newtext_tokens,dim=0)
X = word2vec(tensor_tokenized_newtext,32000) 
embedded_X = embedding_matrix_inputs(X)
positional_embeddingsx = positional_embeddings(X) 
final_X = embedded_X + positional_embeddingsx

In [81]:
from pydub import AudioSegment
from pydub.playback import play
myobj = gTTS(text=cleaned_text, lang='en', slow=False)
myobj.save("welcome.mp3")
from IPython.display import Audio
Audio("welcome.mp3")